In [1]:
import cv2
import glob
import os

def load_points_from_file(file_path):
    """
    指定のtxtファイルから (frame, track, x, y, attr) のタプルを読み込みます。
    """
    points = []
    with open(file_path, 'r') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            parts = line.split(',')
            if len(parts) < 5:
                continue
            frame = int(parts[0])
            track = int(parts[1])
            x = float(parts[2])
            y = float(parts[3])
            attr = parts[4]
            points.append((frame, track, x, y, attr))
    return points

def scale_points_swapped(points, img_width, draw_height):
    """
    各点のx, y座標を反転させます。
    
    - x座標は0～img_widthの範囲で反転させます（例: 0ならimg_width、img_widthなら0）。
    - y座標は0～draw_heightの範囲で反転させます（例: 0ならdraw_height、draw_heightなら0）。
    """
    flipped_points = []
    for point in points:
        frame, track, x, y, attr = point
        flipped_x = img_width - x
        flipped_points.append((frame, track, flipped_x, y, attr))
    return flipped_points


def get_color(attr):
    """
    属性に応じた色を返します。
    属性が 'O' で始まるならピンク、'D' で始まるなら紫を返します。
    """
    if attr.startswith('O'):
        # ピンク
        return (180, 105, 255)
    elif attr.startswith('D'):
        # 紫
        return (128, 0, 128)
    else:
        # 属性が'O'でも'D'でもない場合はデフォルト（黒）を返す
        return (0, 0, 0)

def build_attribute_color_mapping_from_file(gt_file):
    """
    指定のground_truth側のtxtファイルから、属性(attr)→色のマッピング辞書を作成します。  
    同じ属性が出現した場合、最初に出現した属性の色を採用します。
    """
    mapping = {}
    pts = load_points_from_file(gt_file)
    for p in pts:
        _, _, _, _, attr = p
        if attr not in mapping:
            mapping[attr] = get_color(attr)
    return mapping

def draw_points(image, points):
    """
    画像上に各点を大きめの円と拡大された属性テキストで描画します。
    """
    for point in points:
        frame, track, x, y, attr = point
        color = get_color(attr)
        # 座標をintに変換
        x_int, y_int = int(x), int(y)
        # 円の半径を20に、塗りつぶし(-1)
        cv2.circle(image, (x_int, y_int), radius=20, color=color, thickness=-1)
        # テキストの描画位置も整数に変換
        cv2.putText(image, attr, (x_int + 10, y_int - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.5, color, thickness=3)
    return image


def draw_points_with_mapping(image, points, attr_color_mapping):
    """
    画像上に各点を描画します。  
    各点の属性(attr)が attr_color_mapping に存在する場合は、その色で描画し、
    なければ get_color(attr) による色で描画します。  
    円の半径は20、テキストはフォントスケール1.5、太さ3で描画します。
    """
    for point in points:
        frame, track, x, y, attr = point
        if attr in attr_color_mapping:
            color = attr_color_mapping[attr]
        else:
            color = get_color(attr)
        # 座標をintに変換
        x_int, y_int = int(x), int(y)
        # 円の半径を20に、塗りつぶし(-1)
        cv2.circle(image, (x_int, y_int), radius=20, color=color, thickness=-1)
        # テキストの描画位置も整数に変換
        cv2.putText(image, attr, (x_int + 10, y_int - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.5, color, thickness=3)
    return image

In [2]:
# フォルダのパス設定
GT_path = '../../ground_truth/Outdoor/MOT_files/split_transformed/check_ball'
base_image_path = '../../court_images/Outdoor.png'
output_dir_GT = '../../videos/Outdoor/top/minimap'
os.makedirs(output_dir_GT, exist_ok=True)

# 背景画像の読み込みとリサイズ（強制的に縦1050×横1505）
base_image = cv2.imread(base_image_path)
if base_image is None:
    print(f"背景画像の読み込みに失敗しました: {base_image_path}")
else:
    resized_image = cv2.resize(base_image, (1505, 1105))
    height, width = resized_image.shape[:2] 
    print(f"画像サイズ: width={width}, height={height}, 描画領域: {height}px")
    
    txt_files = glob.glob(os.path.join(GT_path, 'IMG_0109_3.txt'))
    if not txt_files:
        print("指定フォルダにtxtファイルが見つかりませんでした:", GT_path)
    
    for txt_file in txt_files:
        print(f"Processing: {txt_file}")
        points = load_points_from_file(txt_file)
        if not points:
            print(f"  {txt_file} には点情報がありません。")
            continue

        # 座標を入れ替え＆固定キャリブレーション値に基づきスケーリング
        scaled_points = scale_points_swapped(points, width, height)
        # フレーム番号ごとに点情報をグループ化
        frames_dict = {}
        for point in scaled_points:
            frame_num = point[0]
            frames_dict.setdefault(frame_num, []).append(point)
        frame_numbers = sorted(frames_dict.keys())
        
        base_filename = os.path.splitext(os.path.basename(txt_file))[0]
        
        # 元動画からFPSを取得
        original_video_path = f'../..//videos/Outdoor/top/split/IMG_0109_3.MOV'
        cap = cv2.VideoCapture(original_video_path)
        fps = cap.get(cv2.CAP_PROP_FPS)
        cap.release()
        if fps <= 0:
            fps = 10  # 取得できなければデフォルト10
        print(f"Using FPS: {fps} for video {base_filename}")
        
        output_video_path = os.path.join(output_dir_GT, f'{base_filename}_GT.mp4')
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        video_writer = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))
        
        for frame_num in frame_numbers:
            frame_img = resized_image.copy()
            frame_points = frames_dict[frame_num]
            # 描画領域内に点と属性テキストを描画
            frame_img = draw_points(frame_img, frame_points)
            # ※ フレーム番号の描画は削除しました
            video_writer.write(frame_img)
        
        video_writer.release()
        print(f"  動画が保存されました: {output_video_path}")


画像サイズ: width=1505, height=1105, 描画領域: 1105px
Processing: ../../ground_truth/Outdoor/MOT_files/split_transformed/check_ball/IMG_0109_3.txt
Using FPS: 29.97002997002997 for video IMG_0109_3
  動画が保存されました: ../../videos/Outdoor/top/minimap/IMG_0109_3_GT.mp4


In [ ]:
# 入力フォルダ設定（BoT-SORT側）
pred_dir = '../../BoT-SORT_outputs/Outdoor/transformed/with_jersey_number/with_team'
output_dir_pred = '../../videos/Outdoor/top/minimap'
os.makedirs(output_dir_pred, exist_ok=True)

# ground_truth側のフォルダ（属性マッピング用）
gt_folder = '../../ground_truth/Outdoor/MOT_files/split_transformed/check_ball'

# 背景画像の読み込みとリサイズ（強制的に縦1105×横1505）
base_image_path = '../../court_images/Outdoor.png'
base_image = cv2.imread(base_image_path)
if base_image is None:
    print(f"背景画像の読み込みに失敗しました: {base_image_path}")
else:
    resized_image = cv2.resize(base_image, (1505, 1105))
    height, width = resized_image.shape[:2]  
    print(f"画像サイズ: width={width}, height={height}, 描画領域: {height}px")
    
    txt_files_pred = glob.glob(os.path.join(pred_dir, 'IMG_0109_3.txt'))
    if not txt_files_pred:
        print("指定フォルダにtxtファイルが見つかりませんでした:", pred_dir)
    
    for txt_file in txt_files_pred:
        print(f"Processing: {txt_file}")
        points = load_points_from_file(txt_file)
        if not points:
            print(f"  {txt_file} には点情報がありません。")
            continue

        # 座標を入れ替え＆固定キャリブレーション値に基づきスケーリング
        scaled_points = scale_points_swapped(points, width, height)
        # フレーム番号ごとに点情報をグループ化
        frames_dict = {}
        for point in scaled_points:
            frame_num = point[0]
            frames_dict.setdefault(frame_num, []).append(point)
        frame_numbers = sorted(frames_dict.keys())
        
        base_filename = os.path.splitext(os.path.basename(txt_file))[0]
        
        # 各BoT-SORTファイルに対応するground_truth側のファイルを探す
        gt_file = os.path.join(gt_folder, base_filename + '.txt')
        if os.path.exists(gt_file):
            attr_color_mapping = build_attribute_color_mapping_from_file(gt_file)
            print(f"Mapping for {base_filename}:", attr_color_mapping)
        else:
            print(f"ground_truth側に対応ファイルが見つかりません: {gt_file}")
            attr_color_mapping = {}
        
        # 元動画からFPSを取得（ファイル名は ./videos/Indoor/{base_filename}.mp4 と仮定）
        original_video_path = f'../../videos/Outdoor/top/split/IMG_0109_3.MOV'
        cap = cv2.VideoCapture(original_video_path)
        fps = cap.get(cv2.CAP_PROP_FPS)
        cap.release()
        if fps <= 0:
            fps = 10  # 取得できなければデフォルト10
        print(f"Using FPS: {fps} for video {base_filename}")
        
        output_video_path = os.path.join(output_dir_pred, f'{base_filename}_pred.mp4')
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        video_writer = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))
        
        for frame_num in frame_numbers:
            frame_img = resized_image.copy()
            frame_points = frames_dict[frame_num]
            # 描画領域内に点と属性テキストを描画（属性マッピングを利用）
            frame_img = draw_points_with_mapping(frame_img, frame_points, attr_color_mapping)
            video_writer.write(frame_img)
        
        video_writer.release()
        print(f"  動画が保存されました: {output_video_path}")


画像サイズ: width=1505, height=1105, 描画領域: 1105px
Processing: ../../BoT-SORT_outputs/Outdoor/transformed/with_jersey_number/with_team/IMG_0109_3.txt
Mapping for IMG_0109_3: {'D10': (128, 0, 128), 'O5': (180, 105, 255), 'O1': (180, 105, 255), 'O3': (180, 105, 255), 'D4': (128, 0, 128), 'D12': (128, 0, 128)}
Using FPS: 29.97002997002997 for video IMG_0109_3
  動画が保存されました: ../../videos/Outdoor/top/minimap/prediction/IMG_0109_3_pred.mp4
